Aprendizado de múltiplas instâncias (MIL):

 é uma abordagem inovadora no campo do aprendizado de máquina que permite trabalhar com conjuntos de dados de forma mais flexível. Em vez de rotular cada instância de forma individual, o MIL rotula conjuntos de instâncias, conhecidos como pacotes.


In [ ]:

# Importar bibliotecas necessárias
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

# Função para criar bags com mais features informativas e instâncias
def create_bags(num_bags=100, num_instances_per_bag=20):
    X, y = [], []
    num_positive_bags = num_bags // 2
    num_negative_bags = num_bags - num_positive_bags

    for _ in range(num_positive_bags):
        # Aumentar o número de features e tornar mais informativas
        bag = make_classification(n_samples=num_instances_per_bag, n_features=20, n_informative=15, n_redundant=5)
        X.append(bag[0])
        y.append(1)

    for _ in range(num_negative_bags):
        bag = make_classification(n_samples=num_instances_per_bag, n_features=20, n_informative=15, n_redundant=5)
        X.append(bag[0])
        y.append(0)

    return X, y

# Gerar os bags
X, y = create_bags()

# Flatten dos dados dos bags para treinar o modelo
X_flat = np.array([instance for bag in X for instance in bag])
y_flat = np.repeat(y, len(X[0]))

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_flat, y_flat, test_size=0.3, stratify=y_flat, random_state=42)

# Definir os parâmetros para otimização com mais regularização
param_grid = {
    'n_estimators': [100],  # Mantemos 100 estimadores
    'max_depth': [10],      # Restringimos a profundidade
    'min_samples_split': [5, 10],  # Aumentar o critério para divisão
    'min_samples_leaf': [4, 6]     # Regularização adicional
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado pelo GridSearchCV
best_model = grid_search.best_estimator_

# Fazer previsões com o melhor modelo
y_pred_best_rf = best_model.predict(X_test)

# Avaliar a acurácia do modelo Random Forest otimizado
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
print(f"Acurácia do modelo Random Forest otimizado: {accuracy_best_rf:.2f}")

# Avaliar o modelo otimizado com ROC AUC
roc_score_best_rf = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
print(f"ROC AUC do modelo Random Forest otimizado: {roc_score_best_rf:.2f}")

# Testar Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

# Treinar o modelo Gradient Boosting
model_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=42)
model_gb.fit(X_train, y_train)

# Fazer previsões com Gradient Boosting
y_pred_gb = model_gb.predict(X_test)

# Avaliar a acurácia e ROC AUC de Gradient Boosting
accuracy_gb = accuracy_score(y_test, y_pred_gb)
roc_score_gb = roc_auc_score(y_test, model_gb.predict_proba(X_test)[:, 1])

print(f"Acurácia do modelo Gradient Boosting: {accuracy_gb:.2f}")
print(f"ROC AUC do modelo Gradient Boosting: {roc_score_gb:.2f}")

# Testar XGBoost
import xgboost as xgb

# Treinar o modelo XGBoost
model_xgb = xgb.XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1, random_state=42)
model_xgb.fit(X_train, y_train)

# Fazer previsões com XGBoost
y_pred_xgb = model_xgb.predict(X_test)

# Avaliar a acurácia e ROC AUC de XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
roc_score_xgb = roc_auc_score(y_test, model_xgb.predict_proba(X_test)[:, 1])

print(f"Acurácia do modelo XGBoost: {accuracy_xgb:.2f}")
print(f"ROC AUC do modelo XGBoost: {roc_score_xgb:.2f}")

"""Desempenho do MIL"""

# Importando as bibliotecas necessárias
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

# Função para criar bags com múltiplas instâncias
def create_bags(num_bags=200, num_instances_per_bag=10):
    X, y = [], []
    # Definir metade dos bags como positivos e a outra metade como negativa
    num_positive_bags = num_bags // 2
    num_negative_bags = num_bags - num_positive_bags

    for _ in range(num_positive_bags):
        # Gerar dados para bags positivos
        bag = make_classification(n_samples=num_instances_per_bag, n_features=10, n_informative=5, n_redundant=3)
        X.append(bag[0])
        y.append(1)

    for _ in range(num_negative_bags):
        # Gerar dados para bags negativos
        bag = make_classification(n_samples=num_instances_per_bag, n_features=10, n_informative=5, n_redundant=3)
        X.append(bag[0])
        y.append(0)

    return X, y

# Criar os bags e instâncias
X, y = create_bags()

# Transformar os dados dos bags em um formato adequado para o modelo
X_flat = np.array([instance for bag in X for instance in bag])
y_flat = np.repeat(y, len(X[0]))

# Dividir os dados entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_flat, y_flat, test_size=0.3, stratify=y_flat, random_state=42)

# Treinando o modelo Random Forest
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = model_rf.predict(X_test)

# Avaliando a acurácia do modelo
accuracy_rf = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo Random Forest: {accuracy_rf:.2f}")

# Avaliando o modelo com ROC AUC
roc_score_rf = roc_auc_score(y_test, model_rf.predict_proba(X_test)[:, 1])
print(f"ROC AUC do modelo Random Forest: {roc_score_rf:.2f}")

"""Adaptação do Algoritmo MIL para Diferentes Cenários


"""

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
import numpy as np

# Função para avaliar com diferentes thresholds
def evaluate_with_threshold(model, X_test, y_test, language, threshold=0.5):
    y_prob = model.predict_proba(X_test)[:, 1]  # Probabilidade da classe positiva
    y_pred = (y_prob >= threshold).astype(int)  # Ajustar o threshold

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Resultados para {language} com threshold {threshold}:")
    print(f"Acurácia: {accuracy:.2f}")
    print(f"Precisão: {precision:.2f}")
    print(f"Revocação: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"Previsões: {y_pred}")
    print(f"Rótulos reais: {y_test}\n")

# Conjunto de dados em português
documents_pt = [
    "Este é um email de spam. Não clique nos links.",
    "Confira esta promoção exclusiva para você.",
    "Envie os documentos para análise.",
    "Preciso de ajuda com o relatório urgente.",
    "Esta mensagem contém malware, evite.",
    "Você foi o escolhido para ganhar este prêmio incrível!",
    "Favor enviar a proposta até sexta-feira.",
    "Sua fatura está atrasada, pague agora para evitar multa.",
    "Esta mensagem é confidencial, não compartilhe.",
    "Parabéns! Você ganhou um desconto exclusivo."
]

# Conjunto de dados em inglês
documents_en = [
    "This is a spam email. Do not click on the links.",
    "Check out this exclusive offer just for you.",
    "Please send the documents for review.",
    "I need help with the urgent report.",
    "This message contains malware, avoid it.",
    "You have been selected to win an amazing prize!",
    "Please submit the proposal by Friday.",
    "Your bill is overdue, pay now to avoid penalty.",
    "This message is confidential, do not share.",
    "Congratulations! You have earned an exclusive discount."
]

# Rótulos dos documentos: 1 para spam, 0 para não spam
labels_pt = [1, 1, 0, 0, 1, 1, 0, 0, 0, 1]
labels_en = [1, 1, 0, 0, 1, 1, 0, 0, 0, 1]

# Usar TF-IDF para representar os documentos em português e inglês separadamente
vectorizer_pt = TfidfVectorizer()
X_pt = vectorizer_pt.fit_transform(documents_pt)

vectorizer_en = TfidfVectorizer()
X_en = vectorizer_en.fit_transform(documents_en)

# Dividir os dados em treino e teste com estratificação para português
X_train_pt, X_test_pt, y_train_pt, y_test_pt = train_test_split(X_pt, labels_pt, test_size=0.3, stratify=labels_pt, random_state=42)

# Dividir os dados em treino e teste com estratificação para inglês
X_train_en, X_test_en, y_train_en, y_test_en = train_test_split(X_en, labels_en, test_size=0.3, stratify=labels_en, random_state=42)

# Treinar o modelo Random Forest
model_rf_pt = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf_pt.fit(X_train_pt, y_train_pt)

model_rf_en = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf_en.fit(X_train_en, y_train_en)

# Avaliar os modelos com diferentes thresholds para português
print("=== Avaliação para Frases em Português ===")
evaluate_with_threshold(model_rf_pt, X_test_pt, y_test_pt, "português", threshold=0.5)  # Threshold padrão
evaluate_with_threshold(model_rf_pt, X_test_pt, y_test_pt, "português", threshold=0.6)  # Threshold ajustado

# Avaliar os modelos com diferentes thresholds para inglês
print("=== Avaliação para Frases em Inglês ===")
evaluate_with_threshold(model_rf_en, X_test_en, y_test_en, "inglês", threshold=0.5)  # Threshold padrão
evaluate_with_threshold(model_rf_en, X_test_en, y_test_en, "inglês", threshold=0.6)  # Threshold ajustado